In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [ ]:
spark = SparkSession.builder\
    .appName('create_stg_tables')\
    .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
    .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enable",True)

In [125]:
####################################################### STAGIN CLIENTS #######################################################

In [4]:
clientes_1 = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep",",") \
  .option("multiline","true") \
  .option("escape", "\\") \
  .load("gs://amazon_bucket_ramiro/stegies/public/stg_clients/2022_02_19_1645304533053_0.csv")

In [5]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import translate
from pyspark.sql.functions import col
from pyspark.sql.functions import split

In [7]:
clientes_2=clientes_1.drop("_airbyte_ab_id","_airbyte_emitted_at") 

In [43]:
clientes_3=clientes_2.withColumn("rowid",clientes_2.rowid.cast('long')) \
    .withColumn("isprime",clientes_2.isprime.cast('boolean')) \
    .withColumnRenamed("id","client_id") \
    .withColumnRenamed("rowid","row_id")

In [48]:
clientes_3.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.stg_clients") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()

In [124]:
####################################################### STAGIN COMPRAS #######################################################

In [108]:
compras_1 = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep",",") \
  .option("multiline","true") \
  .option("escape", "\\") \
  .load("gs://amazon_bucket_ramiro/stegies/public/stg_compras/2022_02_19_1645304533053_0.csv")

In [109]:
compras_2=compras_1.drop("_airbyte_ab_id","_airbyte_emitted_at","rowid")

In [110]:
compras_3=compras_2.withColumn("isprime",compras_2.isprime.cast('boolean')) \
    .withColumnRenamed("id","compra_id")\
    .withColumn("cantidad",compras_2.cantidad.cast("int"))

In [126]:
from google.cloud import storage

In [128]:
client = storage.Client()

In [130]:
source = client.bucket("zophia-proyecto-final-de")

In [131]:
all_blobs = list(client.list_blobs(source))

In [153]:
compras_zophia_2010 =  spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep","|") \
  .option("multiline","false") \
  .option("escape", "\\") \
  .load("gs://zophia-proyecto-final-de/amazon-historico-compras/compras2010.csv")

In [144]:
compras_zophia_2011 =  spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep","|") \
  .option("multiline","false") \
  .option("escape", "\\") \
  .load("gs://zophia-proyecto-final-de/amazon-historico-compras/compras2011.csv")

In [145]:
compras_zophia_2012 =  spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep","|") \
  .option("multiline","false") \
  .option("escape", "\\") \
  .load("gs://zophia-proyecto-final-de/amazon-historico-compras/compras2012.csv")

In [146]:
compras_zophia_2013 =  spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep","|") \
  .option("multiline","false") \
  .option("escape", "\\") \
  .load("gs://zophia-proyecto-final-de/amazon-historico-compras/compras2013.csv")

In [147]:
compras_zophia_2014 =  spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep","|") \
  .option("multiline","false") \
  .option("escape", "\\") \
  .load("gs://zophia-proyecto-final-de/amazon-historico-compras/compras2014.csv")

In [148]:
compras_zophia_2015 =  spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep","|") \
  .option("multiline","false") \
  .option("escape", "\\") \
  .load("gs://zophia-proyecto-final-de/amazon-historico-compras/compras2015.csv")

In [149]:
compras_zophia_2016 =  spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep","|") \
  .option("multiline","false") \
  .option("escape", "\\") \
  .load("gs://zophia-proyecto-final-de/amazon-historico-compras/compras2016.csv")

In [150]:
compras_zophia_2017 =  spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep","|") \
  .option("multiline","false") \
  .option("escape", "\\") \
  .load("gs://zophia-proyecto-final-de/amazon-historico-compras/compras2017.csv")

In [154]:
compras_union=compras_zophia_2010.unionByName(compras_zophia_2017).unionByName(compras_zophia_2016).unionByName(compras_zophia_2015)\
    .unionByName(compras_zophia_2014).unionByName(compras_zophia_2013).unionByName(compras_zophia_2012)\
    .unionByName(compras_zophia_2011)

In [155]:
compras_union_2=compras_union.withColumn("isprime",compras_union.isprime.cast('boolean')) \
    .withColumnRenamed("id","compra_id")\
    .withColumn("cantidad",compras_union.cantidad.cast("int"))

In [156]:
compras_total=compras_union_2.unionByName(compras_3)

In [157]:
compras_total.count()

1965807

In [158]:
compras_total.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.stg_compras") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()

In [ ]:
####################################################### STAGIN EXTERNAL PRODUCTS #######################################################

In [ ]:
external_products = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep",",") \
  .option("multiline","true") \
  .option("escape", "\\") \
  .load("gs://amazon_bucket_ramiro/stegies/public/stg_external_products/2022_02_19_1645304533053_0.csv")

In [ ]:
external_products_2=external_products.drop("_airbyte_ab_id","_airbyte_emitted_at","rowid")

In [ ]:
external_products_3=external_products_2.withColumn("isprime",external_products_2.isprime.cast('boolean')) \
    .withColumn("isbestseller",external_products_2.isbestseller.cast('boolean')) \
    .withColumn("app_sale_price",external_products_2.app_sale_price.cast('float'))
    

In [ ]:
external_products_3.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.stg_external_products") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()

In [ ]:
####################################################### STAGIN PRODUCTS #######################################################

In [209]:
products = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep",",") \
  .option("multiline","true") \
  .option("escape", "\\") \
  .load("gs://amazon_bucket_ramiro/stegies/public/stg_products/2022_02_19_1645304533053_0.csv")

In [210]:
products_2=products.drop("_airbyte_ab_id","_airbyte_emitted_at","rowid") \
    .withColumn("isprime",products.isprime.cast('boolean')) \
    .withColumn("isbestseller",products.isbestseller.cast('boolean'))

In [211]:
products_2.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.stg_products") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()

In [ ]:
####################################################### STAGIN TASAS CAMBIO ANUAL #######################################################

In [190]:
tasas = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("sep",",") \
  .option("multiline","true") \
  .option("escape", "\\") \
  .load("gs://amazon_bucket_ramiro/stegies/public/stg_tasas_cambio_pais_anual/2022_02_19_1645304533053_0.csv")

In [191]:
tasas.show()

+--------------------+-------------------+------------+------------+------------+----+--------+------------------+-----+
|      _airbyte_ab_id|_airbyte_emitted_at|Alpha-2-code|Alpha-3-code|Country-name|Year|currency|             rowid|value|
+--------------------+-------------------+------------+------------+------------+----+--------+------------------+-----+
|06194154-9131-499...|      1645304539289|          AR|         ARG|   Argentina|1962|     ARS|733753960481718274|  0.0|
|1239c22c-626d-44d...|      1645304539289|          AR|         ARG|   Argentina|1963|     ARS|733753960484667394|  0.0|
|5590c1b4-33f3-484...|      1645304539289|          AR|         ARG|   Argentina|1964|     ARS|733753960486993922|  0.0|
|a6babf00-61be-4af...|      1645304539289|          AR|         ARG|   Argentina|1965|     ARS|733753960489025538|  0.0|
|6875b72b-c782-4c4...|      1645304539289|          AR|         ARG|   Argentina|1966|     ARS|733753960490762242|  0.0|
|a261038c-8f28-4fa...|      1645

In [202]:
tasas_2=tasas.drop("_airbyte_ab_id","_airbyte_emitted_at","rowid") \
    .withColumn("value",tasas.value.cast('float')) \
    .withColumnRenamed("Year","currency_year")\
    ..withColumnRenamed("Country_name","country")

In [203]:
tasas_2.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.stg_tasas_cambio_pais_anual") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()